In [ ]:
from jupyter_plotly_dash import JupyterDash

import dash
import dash_leaflet as dl
import dash_core_components as dcc
import dash_html_components as html
import plotly.express as px
import dash_table
from dash.dependencies import Input, Output

import base64
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from pymongo import MongoClient


from AnimalShelter import AnimalShelter



###########################
# Data Manipulation / Model
###########################

username = "aacuser"
password = "password"
shelter = AnimalShelter(username, password)

# class read method must support return of cursor object and accept projection json input
df = pd.DataFrame.from_records(shelter.read({}))

#########################
# Dashboard Layout / View
#########################
app = JupyterDash('SimpleExample')

image_filename = 'Grazioso Salvare Logo.png'
encoded_image = base64.b64encode(open(image_filename, 'rb').read())

app.layout = html.Div([
    html.Div(id='hidden-div', style={'display':'none'}),
    html.Img(
        src='data:image/png;base64,{}'.format(encoded_image.decode()),
        style={
            'height': '200px',
            'width': '200px'
        }),
    html.Center(html.B(html.H1('SNHU CS-340 Dashboard'))),
    html.Center(html.P('JT Bolton, CS-340 Project 2')),
    html.Hr(),
    html.Div(className='row',
        style={'display' : 'flex'},
            children=[
                dcc.Dropdown(
                    id='rescue-dropdown',
                    options=[
                        {'label': 'Water Rescue', 'value': 'water rescue'},
                        {'label': 'Mountain Rescue', 'value': 'mountain rescue'},
                        {'label': 'Wilderness Rescue', 'value': 'wilderness rescue'},
                        {'label': 'Disaster Tracking', 'value': 'disaster tracking'},
                        {'label': 'Individual Tracking', 'value': 'individual tracking'},
                        ],
                    value='all',
                    clearable=False,
                    style={'width': '225px',
                           'textAlign': 'left'}
                ),
                html.Button(id='reset-button', n_clicks=0, children='Reset')
    ]),
    dash_table.DataTable(
        id='datatable-id',
        columns=[
            {"name": i, "id": i, "deletable": False, "selectable": True} for i in df.columns
        ],
        data=df.to_dict('records'),
        style_table={
            'overflowX': 'auto',
        },
        editable=False,
        filter_action="native",
        sort_action="native",
        sort_mode="multi",
        column_selectable=False,
        row_selectable=False,
        row_deletable=False,
        selected_columns=[],
        selected_rows=[0],
        page_action="native",
        page_current= 0,
        page_size= 10,
    ),
    html.Div(id='dd-output-container'),
    html.Br(),
    html.Hr(),
    html.Div(className='row',
        style={'display' : 'flex',
               'width': '100%'},
        children=[
            html.Div([
                dcc.Graph(id='pie-chart')],
                className='col s12 m6',
                style={'width': '50%'}
            ),
        html.Div(
            id='map-id',
            className='col s12 m6',
            style={'width': '50%'}
            )
        ])
    ],
    style={
        'textAlign': 'center',
        'font-family': 'monospace',
})

#############################################
# Interaction Between Components / Controller
#############################################
#This callback will highlight a row on the data table when the user selects it
@app.callback(
    Output('datatable-id', 'style_data_conditional'),
    [Input('datatable-id', 'selected_columns')]
)
def update_styles(selected_columns):
    return [{
        'if': { 'column_id': i },
        'background_color': '#D2F3FF'
    } for i in selected_columns]


#Updates the map when the data table is updated, the first animal in the table is always shown on the map
@app.callback(
    Output('map-id', "children"),
    [Input('datatable-id', "derived_viewport_data")])
def update_map(viewData):
    dff = pd.DataFrame.from_dict(viewData)
    return ([
        dl.Map(center=[dff.iloc[0,13],dff.iloc[0,14]], zoom=10, children=[
            dl.TileLayer(id="base-layer-id"),
            # Marker with tool tip and popup
            dl.Marker(position=[dff.iloc[0,13],dff.iloc[0,14]], children=[
                dl.Tooltip(dff.iloc[0,3]),
                dl.Popup([
                    html.H1("Animal Name"),
                    html.P(dff.iloc[0,9])
                ])
            ])
        ])
    ])

#Pie chart showing the breeds that are currently included in the data table
@app.callback(
    Output("pie-chart", "figure"),
    [Input("datatable-id", "data")])
def generate_chart(data):
    dff = pd.DataFrame.from_dict(data)
    fig = px.pie(
        dff,
        names='breed',
    )
    return fig

#Displays which results are currently being filtered
@app.callback(
    dash.dependencies.Output('dd-output-container', 'children'),
    [dash.dependencies.Input('rescue-dropdown', 'value')])
def update_output(value):
    if (value == None):
        output = 'Showing results for all animals.'
    else:
        output = 'Showing results for {} animals.'.format(value)
    return output

#Dropdown menu that is used to filter results based on type of rescue animal
@app.callback(
    dash.dependencies.Output('datatable-id','data'),
    [dash.dependencies.Input('rescue-dropdown', 'value'),
    ])
def update_table(value):
    if (value == 'all' or value == None):
        df = pd.DataFrame(list(shelter.read({})))
    elif (value == 'water rescue'):
        df = pd.DataFrame(list(shelter.read({'breed': {'$in': ['Labrador Retriever Mix', 'Chesapeake Bay Retriever', 'Newfoundland']},
                                             'sex_upon_outcome': 'Intact Female',
                                             '$and': [{'age_upon_outcome_in_weeks': {'$gte': 26 }}, {'age_upon_outcome_in_weeks': {'$lte': 156}}],                                            
                                            })))
    elif (value == 'mountain rescue' or value == 'wilderness rescue'):
        df = pd.DataFrame(list(shelter.read({'breed': {'$in': ['German Shepherd', 'Alaskan Malamute', 'Old English Sheepdog', 'Siberian Husky', 'Rottweiler']},
                                             'sex_upon_outcome': 'Intact Male',
                                             '$and': [{'age_upon_outcome_in_weeks': {'$gte': 26 }}, {'age_upon_outcome_in_weeks': {'$lte': 156}}],                                            
                                            })))
    elif (value == 'disaster tracking' or value == 'individual tracking'):
        df = pd.DataFrame(list(shelter.read({'breed': {'$in': ['Doberman Pinscher', 'German Shepherd', 'Golden Retriever', 'Bloodhound', 'Rottweiler']},
                                             'sex_upon_outcome': 'Intact Male',
                                             '$and': [{'age_upon_outcome_in_weeks': {'$gte': 20 }}, {'age_upon_outcome_in_weeks': {'$lte': 300}}],                                            
                                            })))
    return df.to_dict('records')

#Resets filter to all animals when the reset button is clicked
@app.callback(Output('rescue-dropdown', 'value'),
             [Input('reset-button', 'n_clicks')])
def reset(n_clicks):
    if n_clicks != 0:
        return 'all'

app